In [ ]:
import os, sys, re
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Setup

In [ ]:
print(sys.executable)
print(sys.version)
print(sys.version_info)
os.environ

In [ ]:
!which jupyter
!jupyter kernelspec list

In [ ]:
!pip install --upgrade pip
!pip install selenium

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--lang=zh")

In [ ]:
driver=webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
base_site_url = "https://www.zzzzzzzzzzz.com/" 

## Get Resource and Download

In [ ]:
def get_video_link_from_91_view_video(driver, viewkey):
    view_video_link = base_site_url +  "/view_video.php?viewkey=" + viewkey
    driver.get(view_video_link)
    html = driver.page_source
    soup = BeautifulSoup(html)

    #get the src attribute from the video tag
    m3u8 = [x.get("src")  for x in soup.findAll("source")]
    title = soup.find('title')

    return m3u8, ' '.join(title.string.split())

## Get viewkey list

In [ ]:
def get_viewkey_links(view_video_link):
    driver.get(view_video_link)
    html = driver.page_source
    soup = BeautifulSoup(html)

    hrefs = [x.get("href")  for x in soup.findAll("a")]
    viewkey_links= [re.search('viewkey=.{20}', x).group().split("=")[1] for x in hrefs if ("viewkey=" in x)]

    return viewkey_links

## Testing

In [ ]:
# testing
m3u8,title = get_video_link_from_91_view_video(driver, "18b04f16a43a9da30315")
print(m3u8, title)
#os.system(f"m3_dl '{m3u8[0]}' -w -o {title}.ts")

## DL for a Page

In [ ]:
viewkey_links=get_viewkey_links(base_site_url + "/v.php?category=hot&viewtype=basic&page=1")
video_meta_list = [get_video_link_from_91_view_video(driver, viewkey) for viewkey in viewkey_links]
video_meta_list[1]

In [ ]:
for v in video_meta_list:
    print(f"Downloading {v[1]}.")
    print(os.system(f"m3_dl '{v[0][0]}' -w -o '{v[1]}.ts' "))